# Generating Transit Travel Times with r5py

In order to calculate access to opportunities, we need to generate transit (or other mode) travel times.

This is a generally computationally intensive process, and has been the main technical hurdle to overcome in this process.

In this workbook, we are going to generate transit travel times using a relativley new Python library `r5py`. R5py is designed to allow Python users to access the open-source R5 engine, a powerful engine that is the spiritual successort to OpenTripPlanner. You can ready more about r5py via [their documentation](https://r5py.readthedocs.io), or check out [R5's own github page](https://github.com/conveyal/r5).
In this workbook we are going to generate transit travel time using the Python library r5py.

In our example analysis, we want to answer two questions:
- How is the access to hospitals distributed across different populations?
- How is the access to child care spaces distributed across different populations?
- How does peak-period and evening service change this access for various populations?

For this we need to generate *two* travel time matrices. One for a peak period (7-9am) and for an evening period (9-11pm).

The beauty of the R5 engine is that it allows us to measure a median peak-period value very easily. To do this, we start our analysis at the beginning of our specified time period (7am and 9pm) and set the duration of the analysis to 120 minutes to cover our 2-hour period.

## Setting Up Data
The first thing we need to do is set up our workbook and data so we can easily run a travel time matrix. To do an analysis we need *origin* and *destination* points. In our case, we are going to keep the calculations relatively minimal by mapping the following origin -> destination travel times:
- Dissemination area centroids -> Hospitals
- Dissemination area centroids -> Child care locations

Let's start by loading the appropriate data and setting some key settings for R5py. One little quirk we are going to need is that R5py requires our origin/destination points to be named `id`, not anything else like `dauid`. We'll make that change now.

In [8]:
import sys
import pandas as pd
import geopandas as gpd

import warnings
warnings.filterwarnings('ignore')
# This sets the amount of memory we are using for R5py calcualtions
sys.argv.append(["--max-memory", "8G"])

da_centroids = gpd.read_file("data/da_centroids_with_locations.geojson").rename(columns={"dauid":"id"})
daycares = gpd.read_file("data/daycare_locations.geojson")
hospitals = gpd.read_file("data/hospital_locations.geojson")

## Set Up Your Transport Network

To calculate travel times, we need to set up a transport network (which happens to be called a `TransportNetwork` class). The transport network needs both an underlying OpenStreetMap PBF file as well as one or more GTFS feeds. So let's go ahead and set up our transport network, which takes as its first argument the path to our PBF file and as a second argument a list of paths to GTFS files (of which we only have one in Calgary).

In [9]:
from r5py import TransportNetwork

transport_network = TransportNetwork(
    "data/Calgary.osm.pbf",
    ["data/cgy-gtfs-2023-03-03.zip"]
)

This will build us a transport network which we can use to compute travel times. So let's go ahead and do that next!
## Computing Travel Times

We create a travel time matrix computer (`TravelTimeMatrixComputer`) which lets us specify a whole bunch of potential parameters, most importantly origins and destinations. Our origins are the DA centroids, and our (first) destination will be the hospital centroids.

**Note:** *The current version of r5py requires that we use the `TransitMode` and `LegMode` objects to specify our modes of travel. Future versions will allow us to just pass along strings like `"WALK"` or `"TRANSIT"`.*

In [10]:
import datetime
from r5py import TravelTimeMatrixComputer, TransitMode, LegMode

travel_time_computer = TravelTimeMatrixComputer(
    transport_network,
    origins=da_centroids,
    destinations=hospitals,
    departure=datetime.datetime(2023, 3, 15, 7, 0),
    departure_time_window=datetime.timedelta(hours=2),
    transport_modes=[TransitMode.TRANSIT, LegMode.WALK]
)

### Compute the Travel Times
Now we're ready to run our computation. This will take a little while to run, and we'll write the matrix directly to a file. In order to avoid writing over the redundant datasets, lets write it to a new file `mx_hospitals_am_workshop.csv`.

In [11]:
travel_time_matrix = travel_time_computer.compute_travel_times()
travel_time_matrix.to_csv("data/mx_hospitals_am_workshop.csv", index=False)

## Verifying our Results
Let's do a quick verification of our results by looking at the minimum travel time to a hospital for each dissemination area and plotting that on a map using the following process:

1. Read in our travel time matrix
2. Group by the origin zone (DA) and take the minimum value
4. Read in our DA areas and join the grouped matrix to it
5. Map it!

Let's do steps 1-3 in the next cell. Null values mean that a hospital is unreachable in the default 2-hour travel time limit we set.

In [5]:
# Step 1
hospital_am = pd.read_csv("data/mx_hospitals_am.csv")
# Step 2
hospital_am = hospital_am[["from_id", "travel_time"]].groupby("from_id", as_index=False).min()
# Need to convert to integer for consistency in joining
hospital_am["from_id"] = hospital_am["from_id"].astype(str)
# Step 3
da_areas = gpd.read_file("data/da_with_locations.geojson")
da_areas = pd.merge(da_areas, hospital_am, left_on="dauid", right_on="from_id")
da_areas.head()

,dauid,daycare,daycare_seats,hospital,geometry,from_id,travel_time
0,48060056,0,0.0,0,"MULTIPOLYGON (((-114.09529 51.13919, -114.0946...",48060056,64.0
1,48060057,0,0.0,0,"MULTIPOLYGON (((-114.09212 51.13986, -114.0934...",48060057,61.0
2,48060058,0,0.0,0,"MULTIPOLYGON (((-114.09502 51.13757, -114.0934...",48060058,61.0
3,48060059,0,0.0,0,"MULTIPOLYGON (((-114.09088 51.13667, -114.0910...",48060059,63.0
4,48060060,0,0.0,0,"MULTIPOLYGON (((-114.08157 51.13836, -114.0826...",48060060,63.0


Now we make a plot, similar to the previous workbook:

In [6]:
import altair as alt

travel_time = alt.Chart(da_areas).mark_geoshape().encode(
    color=alt.Color("travel_time:Q", title="Travel Time (min)", scale=alt.Scale(scheme="plasma"))
).project("mercator")

hosp = alt.Chart(hospitals).mark_circle(size=150, color="white", opacity=1).encode(
    latitude='geometry.coordinates[1]:Q',
    longitude='geometry.coordinates[0]:Q',
    tooltip='facility_name:N'
).project(
    "mercator"
).properties(
    title={
        "text": "Travel Time to the Nearest Hospital",
        "subtitle": "Morning Peak (7-9am) on Wednesday, March 15, 2023"
    },
    width=700,
    height=900
)

travel_time + hosp

alt.LayerChart(...)